# Implementation of a Autoencoder (AE)
#### based on PyTorch tutorial (https://pytorch.org/tutorials/beginner/basics/intro.html)
## Simple AE with one layer into latent space 748 -> 10
##### I hope it also does it backwards (at least it works, but other implementations explicitly specify the backward layer and this one does not (in NeuralNetwork_AE)).
#### Build for MINST datasets

###### ***For sc-RNAseq:*** fix enable cuda/GPU (only needed if CPU is to slow), try different optimiser, check for loss function, trying learning rates, nn.ReLU() function may not be the best, checking for overfitting by plotting loss of model and training data, Batch size has to be optimised: https://arxiv.org/abs/1609.04836 , https://arxiv.org/abs/1703.04933 . Maybe we should try a program like https://opendatascience.com/optimizing-pytorch-performance-batch-size-with-pytorch-profiler/ for that on a later stage for performance (not sure if the data will be to large in the future)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
batch_size = 64

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork_AE(nn.Module):
    def __init__(self):
        super(NeuralNetwork_AE, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork_AE() #.to(device) #### this is needed for cuda
print(model)

NeuralNetwork_AE(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
learning_rate = 5e-4
epochs = 5

In [7]:
# loss function:
    # nn.MSELoss (Mean Square Error) for regression tasks
    # nn.NLLLoss (Negative Log Likelihood) for classification
    # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss


loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.317503  [    0/60000]
loss: 0.395707  [ 6400/60000]
loss: 0.263923  [12800/60000]
loss: 0.332832  [19200/60000]
loss: 0.214908  [25600/60000]
loss: 0.307582  [32000/60000]
loss: 0.135500  [38400/60000]
loss: 0.330453  [44800/60000]
loss: 0.257580  [51200/60000]
loss: 0.294584  [57600/60000]
Test Error: 
 Accuracy: 94.4%, Avg loss: 0.185234 

Epoch 2
-------------------------------
loss: 0.118693  [    0/60000]
loss: 0.191264  [ 6400/60000]
loss: 0.099930  [12800/60000]
loss: 0.192193  [19200/60000]
loss: 0.140712  [25600/60000]
loss: 0.218253  [32000/60000]
loss: 0.057894  [38400/60000]
loss: 0.227659  [44800/60000]
loss: 0.180948  [51200/60000]
loss: 0.206923  [57600/60000]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.128071 

Epoch 3
-------------------------------
loss: 0.076911  [    0/60000]
loss: 0.126406  [ 6400/60000]
loss: 0.077345  [12800/60000]
loss: 0.097515  [19200/60000]
loss: 0.092595  [25600/60000]
loss: 0.142932  [32000/600

In [9]:
torch.save(model, 'model_oneLayer_AE.pth')

In [10]:
model = torch.load('model_oneLayer_AE.pth')

In [11]:
model

NeuralNetwork_AE(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)